In [5]:
import requests
import pandas as pd

BASE_URL = "https://fantasy.premierleague.com/api"
LEAGUE_ID = 16511  # Replace with actual league ID

# Function to get list of managers in the league
def get_league_entries(league_id):
    entries = []
    page = 1
    while True:
        url = f"{BASE_URL}/leagues-classic/{league_id}/standings/?page_new_entries=1&page_standings={page}"
        r = requests.get(url)
        data = r.json()
        standings = data['standings']['results']
        if not standings:
            break
        entries.extend(standings)
        page += 1
    return entries

# Function to get weekly history for a given manager/team ID
def get_team_weekly_data(entry_id):
    url = f"{BASE_URL}/entry/{entry_id}/history/"
    r = requests.get(url)
    if r.status_code != 200:
        return None
    data = r.json()
    return data.get('current', [])  # Weekly data

# Main function to gather all weekly data
def get_league_weekly_data(league_id):
    league_entries = get_league_entries(league_id)
    all_data = []

    for entry in league_entries:
        entry_id = entry['entry']  # FPL team ID
        player_name = entry['player_name']
        team_name = entry['entry_name']
        weekly_data = get_team_weekly_data(entry_id)
        for gw in weekly_data:
            gw['player_name'] = player_name
            gw['team_name'] = team_name
            gw['entry_id'] = entry_id
            all_data.append(gw)

    return pd.DataFrame(all_data)

# Example usage
df = get_league_weekly_data(LEAGUE_ID)
df['player_name'] = df['player_name'].str.title()

In [6]:
(df[df["event"]==3]).sort_values(by='points', ascending=False)

,event,points,total_points,rank,rank_sort,overall_rank,percentile_rank,bank,value,event_transfers,event_transfers_cost,points_on_bench,player_name,team_name,entry_id
382,3,87,213,489783,501442,2157276,5,6,1006,0,0,9,Rahul Mahida,The Usual Suspects,2804590
496,3,87,203,579122,579669,3304964,10,2,1000,4,8,5,Bharesh Patel,Boz Mania,3154674
686,3,85,211,695156,700047,2365476,10,2,1004,0,0,2,Ravi Kotecha,Kai's SakaWhiteRice,784000
192,3,84,214,874744,888309,2054519,10,15,1000,1,0,2,Amit Shah,Toney Soprano,2605199
154,3,82,228,1107888,1119211,856457,15,0,1001,0,0,2,Simon Brawn,Hoping for £10 �,1480436
116,3,81,224,1280626,1289059,1140088,15,1,1006,0,0,12,Prish Roy,BanJota,902365
458,3,81,229,1357687,1390692,792765,15,0,1002,1,0,1,Jameel Shah,"Cesc, Drogs & ACole",4701430
534,3,80,210,1472384,1492264,2497137,15,4,1003,0,0,1,Tom Sharp,sharp shooters,2404917
572,3,80,217,1651334,1652927,1827960,20,0,1005,3,8,3,Akshay Loomba,Feeling SlotSlotSlot,1328345
78,3,79,208,1762065,1762093,2711539,20,24,1004,1,0,2,Dan Coulton,Coult-on the Break,1582


In [7]:
df_max_per_event = df[df['points'] == df.groupby('event')['points'].transform('max')]
df_max_per_event.sort_values(by='event', ascending=True)

,event,points,total_points,rank,rank_sort,overall_rank,percentile_rank,bank,value,event_transfers,event_transfers_cost,points_on_bench,player_name,team_name,entry_id
798,1,84,84,152756,166230,152756,5,0,1000,0,0,3,Ravi Patel,Mid Table Madmen,3118163
647,2,87,152,1291466,1293165,917196,15,0,1001,2,4,3,Cedric Mendes,Gnad mei Lnud,1239099
496,3,87,203,579122,579669,3304964,10,2,1000,4,8,5,Bharesh Patel,Boz Mania,3154674
382,3,87,213,489783,501442,2157276,5,6,1006,0,0,9,Rahul Mahida,The Usual Suspects,2804590
117,4,70,294,675081,678167,349635,10,1,1009,0,0,10,Prish Roy,BanJota,902365
156,5,84,369,252384,253813,194877,5,2,1011,2,0,2,Simon Brawn,Hoping for £10 �,1480436
765,6,78,396,692627,693270,1104158,10,0,1014,2,0,9,Fabian Carvalho,KayFabe XI,333762
424,7,94,474,4656,4657,191536,1,15,1020,1,0,6,Samit R Patel,Token entry,91924
501,8,69,468,39108,39556,2021712,1,11,1005,2,0,0,Bharesh Patel,Boz Mania,3154674
312,9,76,551,409604,419161,857664,5,0,1015,1,0,6,Mihir Patel,M-UNIT,3749756


In [8]:
event_counts = df_max_per_event.groupby('event')['player_name'].transform('count')

# Step 2: Assign fractional credit to each row
df_max_per_event['score_share'] = 1 / event_counts

# Step 3: Group by player_name and sum the fractional scores
player_scores = df_max_per_event.groupby('player_name')['score_share'].sum().reset_index()

# Optional: sort by score
player_scores = player_scores.sort_values(by='score_share', ascending=False)

print(player_scores)

        player_name  score_share
3     Cedric Mendes          4.5
17      Simon Brawn          3.5
4       Dan Coulton          3.0
1         Amit Shah          3.0
11        Prish Roy          2.5
2     Bharesh Patel          2.5
16    Shivaan Popat          2.0
13     Ravi Kotecha          2.0
12     Rahul Mahida          2.0
5   Fabian Carvalho          2.0
15    Samit R Patel          2.0
0        Amit Kerai          1.5
8      Josh Thomson          1.5
7     Jaymesh Patel          1.5
9       Mihir Patel          1.5
6       Jameel Shah          1.0
10      Mitesh Shah          1.0
14       Ravi Patel          1.0


C:\Users\danie\AppData\Local\Temp\ipykernel_11916\1906090238.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_max_per_event['score_share'] = 1 / event_counts
